In [ ]:
#Step1 system prompt: You're a test engineer, ...
# step2: instantiate openAI
# Step3: add dq.md as a knowledge to chatgpt
# Step4: User prompt- run the data profiler and generate the report as sample data for testing


In [ ]:
import os
import json
import pandas as pd
from openai import OpenAI    # ensure you have the correct OpenAI client installed
from dotenv import load_dotenv


In [ ]:
# ---------- CONFIG ----------
DQ_PATH = "/Users/somyak/projects/llm_engineering/keyrus_agentic_testing_framework/data_profiler_tool/dq.md"               # path to your dq.md file you created
DATA_PATH = "/Users/somyak/projects/llm_engineering/keyrus_agentic_testing_framework/dummy_data_profiler_1000000_rows.csv"     # path to your dummy master data (update if different)
OUTPUT_PATH = "/Users/somyak/projects/llm_engineering/keyrus_agentic_testing_framework/data_profiler_tool"
MODEL = "gpt-4.1-nano"

In [ ]:
# ----------Load environment variables in a file called .env--------------------------
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
openai = OpenAI()

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

In [ ]:
# ---------- helper: system prompt ----------
SYSTEM_PROMPT = """You are an AI Data Test Engineer designed to automate the end-to-end data profiling workflow for the PADS project.

Your responsibilities:
1. Load and use all Data Quality Rules contained in the knowledge file 'dq.md' that I uploaded.
2. Understand the workflow:
   - Input: raw master dataset (UAT extract).
   - After import to SQL, three columns are added: RecordId, CreationDate, {TLA}_Filter.
   - Profiling is applied on the enriched master table.
3. Apply every DQ rule from dq.md (null checks, duplicates, datatypes, allowed values, cross-column rules, range/quartiles/outliers).
4. Run variance profiling according to dq.md (ALL, RANGE, TOPVALUE, DISTINCT, POSITIVENEGATIVE when applicable).
5. Compute ValueCount, ValueDistribution (pct), VarianceHitCount, VarianceHitRank, VarianceHitDenseRank, Quartile, SumValueDistribution, and select representative RecordId(s) for testing.
6. Output: (a) a brief profiling summary, and (b) the final sample dataset (rows) — exported as CSV rows where each row is a record from the original master data.
7. NEVER invent new rules — always use dq.md as the source of truth.

Important:
- For the large dataset, you can read the top N rows locally for context, but you must base decisions on the full dataset.
- Provide the final sample data as CSV text (so it can be parsed and saved).
"""

In [ ]:

# ---------- helper: user prompt (concise) ----------
USER_PROMPT = f"""
I have uploaded a DQ rules file at this local path (on the same host): {DQ_PATH}
I have a master dataset at: {DATA_PATH}

Please:
 1) Load the DQ rules from the file at the local path above.
 2) Apply those rules to the dataset at the local path above.
 3) Add RecordId, CreationDate, POLICY_Filter if missing (RecordId sequential).
 4) Compute profiling metrics and pick ~30-50 representative sample rows.
 5) Output: (A) a short profiling summary, and (B) the final sample CSV between
    markers: ===SAMPLE_CSV=== (start) and ===END_SAMPLE_CSV=== (end).
Return only the summary followed by the CSV block. The platform will transform the local path to a URL and make that file available to you; use that file as the single source of truth for DQ rules.
"""

In [ ]:
print("Sending request to agent... (may take some seconds)")

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": USER_PROMPT}
]

response = openai.chat.completions.create(
    model="gpt-4.1-nano",
    messages=messages,
    files=[
        {"file_url": DQ_PATH},
        {"file_url": DATA_PATH}
    ]
)

assistant_text = response.choices[0].message["content"]
print(assistant_text)


In [ ]:
from openai import OpenAI
import os
import time
import json

client = OpenAI()
MODEL = "gpt-4.1-nano"

DQ_PATH = "/Users/somyak/projects/llm_engineering/keyrus_agentic_testing_framework/data_profiler_tool/dq.md"
DATA_PATH = "/Users/somyak/projects/llm_engineering/keyrus_agentic_testing_framework/dummy_data_profiler_1000000_rows.csv"
OUTPUT_CSV = "/Users/somyak/projects/llm_engineering/keyrus_agentic_testing_framework/data_profiler_tool/profiler_sample_output_from_agent.csv"

SYSTEM_PROMPT = "You are an AI Data Test Engineer. Use ONLY the DQ rules file provided in the FILE_URLS block. Do not invent rules."
USER_INTENT = (
    "Please load the files listed in the FILE_URLS block (they are local paths on the host), "
    "apply the dq.md rules to the master CSV, and return a final sample CSV between markers."
)

# Structured file-list message - platform will detect these local paths and host them for the agent/tool.
file_list_message = (
    "FILE_URLS:\n"
    f"- {DQ_PATH}\n"
    f"- {DATA_PATH}\n\n"
    "The platform will transform these local paths into hosted URLs the agent can access. "
    "Now: perform the profiling as specified below."
)

USER_PROMPT = (
    file_list_message + "\n\n"
    + "TASK:\n"
    "1) Load the DQ rules from the dq.md file path above.\n"
    "2) Load the master dataset from the CSV path above.\n"
    "3) If missing, add RecordId (sequential), CreationDate (today), and POLICY_Filter='ALL'.\n"
    "4) Apply every DQ rule from dq.md, compute profiling metrics, variance groups, quartiles, etc.\n"
    "5) Select ~30-50 representative records covering top/mid/low frequency buckets and Premium ranges.\n"
    "6) Output:\n"
    "   - A short profiling summary, then\n"
    "   - The final sample CSV **only** between these markers:\n"
    "     ===SAMPLE_CSV===\n"
    "     <csv header + rows>\n"
    "     ===END_SAMPLE_CSV===\n"
    "Return nothing after the end marker."
)

def call_agent_with_retries(max_attempts=5, pause_seconds=8):
    for attempt in range(1, max_attempts + 1):
        print(f"[attempt {attempt}] sending request to agent...")
        resp = client.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": USER_PROMPT}
            ],
            temperature=0.0,
            max_tokens=8000
        )

        # extract assistant content robustly
        try:
            assistant_text = resp.choices[0].message["content"]
        except Exception:
            assistant_text = str(resp)

        # quick diagnostic
        preview = assistant_text[:800].strip()
        print("assistant preview:", preview.replace("\n", " ")[:300])

        # If assistant returned only an acknowledgement ("I will do X"), retry after pause
        ack_keywords = [
            "i will now proceed",
            "please hold on",
            "i will proceed",
            "processing these steps",
            "i will now process"
        ]
        lower = assistant_text.lower()
        if any(k in lower for k in ack_keywords):
            if attempt < max_attempts:
                print("Assistant acknowledged but did not return CSV. Waiting and retrying...")
                time.sleep(pause_seconds)
                continue
            else:
                print("Max attempts reached. Assistant never returned CSV. Raw response:\n")
                print(assistant_text[:4000])
                return None

        # Try to extract CSV between markers
        start = "===SAMPLE_CSV==="
        end = "===END_SAMPLE_CSV==="
        if start in assistant_text and end in assistant_text:
            csv_block = assistant_text.split(start, 1)[1].split(end, 1)[0].strip()
            if "," in csv_block and "\n" in csv_block:
                print("CSV found. Saving to:", OUTPUT_CSV)
                with open(OUTPUT_CSV, "w", encoding="utf-8") as f:
                    f.write(csv_block)
                return OUTPUT_CSV
            else:
                print("Found markers but content does not look like CSV. Dumping snippet:")
                print(csv_block[:1000])
                return None
        else:
            # No markers and no obvious ACK => print a bit and retry once
            if attempt < max_attempts:
                print("No CSV markers found. Retrying after short pause...")
                time.sleep(pause_seconds)
                continue
            else:
                print("No CSV markers after retries. Final assistant output (truncated):\n")
                print(assistant_text[:4000])
                return None

# Run
result_path = call_agent_with_retries(max_attempts=6, pause_seconds=10)
if result_path:
    print("Agent-produced CSV available at:", result_path)
else:
    print("Agent did not return CSV. See logs above for assistant output.")
